## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
import matplotlib.pyplot as plt

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Bouna,CI,9.2667,-3.0000,72.82,89,81,0.98,light rain
1,1,Khatanga,RU,71.9667,102.5000,61.92,40,100,10.78,overcast clouds
2,2,Ushuaia,AR,-54.8000,-68.3000,42.46,93,40,11.50,scattered clouds
3,3,Salalah,OM,17.0151,54.0924,86.09,84,75,5.75,broken clouds
4,4,Sorong,ID,-0.8833,131.2500,77.85,88,100,7.65,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Salalah,OM,17.0151,54.0924,86.09,84,75,5.75,broken clouds
4,4,Sorong,ID,-0.8833,131.2500,77.85,88,100,7.65,overcast clouds
15,15,Puerto Ayora,EC,-0.7393,-90.3518,78.76,92,51,7.00,broken clouds
16,16,Kapaa,US,22.0752,-159.3190,84.18,75,20,16.11,few clouds
20,20,Nipawin,CA,53.3668,-104.0009,78.84,31,75,5.75,broken clouds
22,22,Isla Mujeres,MX,21.2311,-86.7310,86.16,76,4,10.45,clear sky
27,27,Vaini,TO,-21.2000,-175.2000,80.76,89,40,9.22,scattered clouds
28,28,Victoria,HK,22.2855,114.1577,84.18,87,100,11.99,overcast clouds
37,37,Hithadhoo,MV,-0.6000,73.0833,83.62,73,100,9.04,overcast clouds
40,40,Bathsheba,BB,13.2167,-59.5167,83.01,78,20,19.57,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df

clean_df.dropna(axis=0, how='any',inplace=True)

C:\Users\Ian\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [7]:
clean_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Salalah,OM,86.09,broken clouds,17.0151,54.0924,
4,Sorong,ID,77.85,overcast clouds,-0.8833,131.2500,
15,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
16,Kapaa,US,84.18,few clouds,22.0752,-159.3190,
20,Nipawin,CA,78.84,broken clouds,53.3668,-104.0009,
22,Isla Mujeres,MX,86.16,clear sky,21.2311,-86.7310,
27,Vaini,TO,80.76,scattered clouds,-21.2000,-175.2000,
28,Victoria,HK,84.18,overcast clouds,22.2855,114.1577,
37,Hithadhoo,MV,83.62,overcast clouds,-0.6000,73.0833,
40,Bathsheba,BB,83.01,few clouds,13.2167,-59.5167,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]


    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.replace('',np.nan,regex=True)

clean_hotel_df.dropna(axis=0,how='any',inplace=True)

clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Salalah,OM,86.09,broken clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
4,Sorong,ID,77.85,overcast clouds,-0.8833,131.2500,Waigo Splash Hotel
15,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
16,Kapaa,US,84.18,few clouds,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
20,Nipawin,CA,78.84,broken clouds,53.3668,-104.0009,Tobin Lake Motel


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Description}</dd>
<dt>Max Temperature</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [13]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
hotel_fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
hotel_marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

hotel_fig.add_layer(hotel_marker_layer)

# 11b. Display the figure
hotel_fig

Figure(layout=FigureLayout(height='420px'))